# Mocule for Molecular Dynamics.

In [2]:
#!/bin/env python
import os
import sys
import time


In [ ]:
class LigParGen():
    def __init__(self, is_headless=True):

        self.url = 'http://zarbi.chem.yale.edu/ligpargen/'
        self.prefs = {"download.default_directory" : os.getcwd()}


    def get_opls_from_smiles(self,
            smiles='CCC',
            format='lammps',
            to='filename'
    ):
        import selenium
        from selenium import webdriver
        from selenium.webdriver.common.keys import Keys
        self.options = webdriver.ChromeOptions()
    #    options.add_argument('--headless')
        self.options.add_experimental_option('prefs', self.prefs)
        if is_headless: 
            self.options.add_argument('--headless')
        self.driver = webdriver.Chrome(options=self.options)
        
        self.driver.get(self.url)
        
        print(self.driver.title)
        box = self.driver.find_element_by_id('smiles')
        print(box)
        box.send_keys(smiles)

        time.sleep(10)

        b = self.driver.find_element_by_class_name("btn")
        print(b)
        b.submit()

        time.sleep(10)

        p = self.driver.find_elements_by_css_selector('p')
        print(p)
        f = self.driver.find_element_by_class_name('form-group')
        keys = [_.get_property('value') for _ in  f.find_elements_by_name('go')]
        d = dict(list(zip(keys, f.find_elements_by_name('go'))))
        d['LAMMPS'].submit()

        time.sleep(10)

In [5]:
def test():
    send_smiles_ligpargen()

LigParGen Server
<selenium.webdriver.remote.webelement.WebElement (session="d15ba799faf21c4fbf00d0ac425fbb73", element="12ef019e-da32-4498-bc89-eba8014a4650")>
<selenium.webdriver.remote.webelement.WebElement (session="d15ba799faf21c4fbf00d0ac425fbb73", element="f6bb061e-1b27-4413-8181-f965a16357db")>
[<selenium.webdriver.remote.webelement.WebElement (session="d15ba799faf21c4fbf00d0ac425fbb73", element="cd311add-d96c-4ac3-b10c-ef5e8842e063")>, <selenium.webdriver.remote.webelement.WebElement (session="d15ba799faf21c4fbf00d0ac425fbb73", element="fa20b8bc-cb97-41fd-b02a-af73a37f1b6e")>, <selenium.webdriver.remote.webelement.WebElement (session="d15ba799faf21c4fbf00d0ac425fbb73", element="8809861f-e111-4824-8337-e2f00a0303c7")>, <selenium.webdriver.remote.webelement.WebElement (session="d15ba799faf21c4fbf00d0ac425fbb73", element="e9f12033-f275-48ca-8d5b-2f4f2d9463e7")>, <selenium.webdriver.remote.webelement.WebElement (session="d15ba799faf21c4fbf00d0ac425fbb73", element="d56f8f0f-3775-48f